<a href="https://www.kaggle.com/code/gabrielbchacon/classification-approach-detecting-fraud?scriptVersionId=94153874" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
#Importing Pandas and NumPy <3
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

#Importing SkLearn modules
from sklearn.compose import ColumnTransformer 
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/fraudulent-transactions-prediction/Fraud.csv


# IMPORTING DATASET WITH PANDAS

In [2]:
dataframe = pd.read_csv('/kaggle/input/fraudulent-transactions-prediction/Fraud.csv')
dataset = dataframe.copy()

In [3]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [4]:
missing = dataset.isna().sum()
missing

step              0
type              0
amount            0
nameOrig          0
oldbalanceOrg     0
newbalanceOrig    0
nameDest          0
oldbalanceDest    0
newbalanceDest    0
isFraud           0
isFlaggedFraud    0
dtype: int64

Looks like this dataset doesn't have null values, so it resumes my cleaning step

# PREPROCESSING 


In [5]:
#First, converting step into real time hours accoding the definition:
## maps a unit of time in the real world. In this case 1 step is 1 hour of time. Total steps 744 (30 days simulation).
##Here considering 744/24 = 31, I considered 31 days of simulation. 

dataset['time_hour'] = round(dataset['step']/31)

In [6]:
# generate binary values using get_dummies for the categorical features: Type and Time_hour

dataset = pd.get_dummies(dataset, columns=["type", 'time_hour'])

Dropping columns I'll not use:

All the columns related to the custumer ID. Even it may be related in long term, like the same person doing a lot of frauds, I think the criminals always will try to use other usersID to avoid this kind of blocks. So let's find another behavior related. 

Also I dropped the balance because it looks not be related in the previous analises. 

It was dropped the last columns of the dummies features to avoid over fitting. 



In [7]:
columns_to_drop = ['nameOrig', 'nameDest', 'type_TRANSFER', 'oldbalanceOrg', 'newbalanceOrig', 'oldbalanceDest', 'newbalanceDest', 'isFlaggedFraud', 'step', 'time_hour_24.0']
dataset = dataset.drop(columns = columns_to_drop)

In [8]:
#Using StandartScaler to minimize the importance of amount. 

transformer = ColumnTransformer(transformers=[('scaler', StandardScaler(), ['amount'])])
transformer.fit_transform(dataset)

array([[-0.28155992],
       [-0.29476726],
       [-0.2975548 ],
       ...,
       [10.1539526 ],
       [ 1.1097649 ],
       [ 1.1097649 ]])

In [9]:
dataset['amount'] = transformer.fit_transform(dataset)

In [10]:
#This specific dataset is huge! So let's use just 30000 random lines to train the model. It will resume the execution time.  
dataset_model = dataset.sample(n=30000)

In [11]:
#Spliting the X and y

y = dataset_model['isFraud']
X = dataset_model[['amount', 'type_CASH_IN', 'type_CASH_OUT', 'type_DEBIT',
       'type_PAYMENT', 'time_hour_0.0', 'time_hour_1.0', 'time_hour_2.0',
       'time_hour_3.0', 'time_hour_4.0', 'time_hour_5.0', 'time_hour_6.0',
       'time_hour_7.0', 'time_hour_8.0', 'time_hour_9.0', 'time_hour_10.0',
       'time_hour_11.0', 'time_hour_12.0', 'time_hour_13.0', 'time_hour_14.0',
       'time_hour_15.0', 'time_hour_16.0', 'time_hour_17.0', 'time_hour_18.0',
       'time_hour_19.0', 'time_hour_20.0', 'time_hour_21.0', 'time_hour_22.0',
       'time_hour_23.0']]

# Model Training

In [12]:
#spliting data into train and test samples
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

In [13]:
#model fitting using SVC
model = SVC()
model.fit(X_train, y_train)

SVC()

In [14]:
#Checking the accuracy for my model
predict = model.predict(X_test)
accuracy_score(y_test, predict)

0.9989166666666667

This accuracy looks very good (almost perfect), which may be saying that I did something wrong. 